In [5]:
# read it to inspect it
with open('/Users/js222/Desktop/LLM_related/data/tinyshakespeare/input.txt', 'r', encoding='utf-8') as file:
    text = file.read()

In [6]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [7]:
# check the first 1000 chars
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [8]:
# all unique chars in the text
chars  = sorted(list(set(text)))
vocab_size = len(chars)
print(f"All chars: {''.join(chars)}")
print(f"vocan_size: {vocab_size}")

All chars: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocan_size: 65


In [11]:
# create a mapping from chars to integers
stoi = {c:i for i, c in enumerate(chars)}
itos = {i:c for i, c in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] # encode: take a tring, output a list of integers
decode = lambda i: ''.join([itos[idx] for idx in i]) # decode: take a list of integers, output a string

In [12]:
print(encode("hello how are you"))
print(decode(encode("hello how are you")))

[46, 43, 50, 50, 53, 1, 46, 53, 61, 1, 39, 56, 43, 1, 63, 53, 59]
hello how are you


In [13]:
# encode the entire dataset and store it into a torch.tensor
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [35]:
# split up the data into train and validation sets
n = int(0.9 * len(text)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [15]:
# block_size可以理解为上下文窗口
block_size = 8
train_data[:block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [16]:
x = train_data[:block_size]
y = train_data[1:block_size + 1]
for t in range(block_size):
    context = x[:t + 1]
    target = y[t]
    print(f"when input in {context}, the target: {target}")

when input in tensor([18]), the target: 47
when input in tensor([18, 47]), the target: 56
when input in tensor([18, 47, 56]), the target: 57
when input in tensor([18, 47, 56, 57]), the target: 58
when input in tensor([18, 47, 56, 57, 58]), the target: 1
when input in tensor([18, 47, 56, 57, 58,  1]), the target: 15
when input in tensor([18, 47, 56, 57, 58,  1, 15]), the target: 47
when input in tensor([18, 47, 56, 57, 58,  1, 15, 47]), the target: 58


In [32]:
torch.randint(10, (2,))

tensor([1, 2])

In [36]:
# build batch
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will be processed in parallel
block_size = 8 # maximum context length for predictions

def get_batch(split: str):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(train_data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print("inputs:")
print(xb.shape)
print(xb)
print("targets:")
print(yb.shape)
print(yb)


inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
